In [1]:
!pip install ampligraph==1.4.0
!pip install tensorflow==1.13.1
# Do not run if these are already installed

**** Missing build for a loaded module for Python version "3.7.10". 
Directory does not exist: /share/pkg.7/tensorflow/1.13.1/install/lib/site-packages/../python3.7/site-packages

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/

In [6]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import ampligraph

In [7]:
# Load CWE Views, Categories and Hierarcical relationships (parent, peer..)
cwe_view = pd.read_csv("../datasets/processed/csv_file/2021aug/cwe_view.csv")
cwe_category = pd.read_csv("../datasets/processed/csv_file/2021aug/cwe_category.csv")
cwe_rels = pd.read_csv("./cwe_relations.csv")

In [8]:
views = []

for i, item in cwe_view.iterrows():
    views.append([item['ID'],item['Name'],item['Has_Member']])

In [9]:
categories = []

for i, item in cwe_category.iterrows():
    categories.append([item['ID'],item['Name'],item['Has_Member']])

In [10]:
df_cve = pd.read_csv("../datasets/processed/csv_file/2021aug/cve/cve-2002.csv")
df_cve.fillna('*', inplace = True)
for i in range(2003, 2022):
    filename = '../datasets/processed/csv_file/2021aug/cve/cve-' + str(i) + '.csv'
    df_temp = pd.read_csv(filename)
    df_temp.fillna('*', inplace = True)
    df_cve=pd.concat([df_cve, df_temp])

In [11]:
df_cwe = pd.read_csv("../datasets/processed/csv_file/2021aug/cwe.csv")
df_cwe.fillna('*', inplace=True)

df_cwe_cate = pd.read_csv("../datasets/processed/csv_file/2021aug/cwe_category.csv")
df_cwe_view = pd.read_csv("../datasets/processed/csv_file/2021aug/cwe_view.csv")
df_cwe_cate.fillna('*', inplace=True)
df_cwe_view.fillna('*', inplace=True)

In [40]:
df_cwe_1003 = pd.read_csv("./cwe-1003.csv", usecols = ['CWE-ID','Weakness Abstraction'], index_col=False)
df_cwe_1003.fillna('*', inplace=True)
df_cwe_1003.columns

Index(['CWE-ID', 'Weakness Abstraction'], dtype='object')

In [42]:
cwe_sublist = []
cwe_1003 = []
cwe_disc = ["CWE-119","CWE-20","CWE-200","CWE-269","CWE-289",
"CWE-311","CWE-330","CWE-345","CWE-74","CWE-400","CWE-610","CWE-662",
"CWE-665","CWE-668","CWE-682","CWE-697","CWE-755","CWE-834"]
for i,r in df_cwe_1003.iterrows():
    cweid = r['CWE-ID']
    cwe_abs = r['Weakness Abstraction']
    cwe_1003.append(cweid)
    
    if cweid not in cwe_disc:
        cweid = "CWE-" + str(cweid)
        cwe_sublist.append((cweid,cwe_abs))
        
#print(cwe_sublist)

In [12]:
cwe_list = []

kg_cwe = []

for i,r in df_cwe.iterrows():
    cweid = r['ID']
    cwe_list.append(cweid)

In [15]:
kg_cvecwe_all = []

for i,r in df_cve.iterrows():
    cveid = r['ID']

    if (r['MatchingCWE'] != '*' and r['MatchingCWE'] != 'NVD-CWE-Other' and r['MatchingCWE'] != 'NVD-CWE-noinfo'):
        cwelist = r['MatchingCWE'].split(';')
        for w in cwelist:
            matchcwe = [cveid, 'MatchingCWE', w]
            kg_cvecwe_all.append((matchcwe))
    else:
        matchcwe = [cveid, 'MatchingCWE', r['MatchingCWE']]
        kg_cvecwe_all.append((matchcwe))

In [16]:
df_cvecwe2 = pd.DataFrame(kg_cvecwe_all, columns = ['CVE', 'Predicate', 'CWE'])

In [18]:
# Dataframe of all CVE-CWE mappings, including cwe-noinfo and cwe-other. 
df_cvecwe2.to_csv('df_cve2cwe.csv', index=False)

For each view and category, and each CVE that are mapped to those, we query the KG for valid CWE mappings from the members of the same view or category

In [19]:
'''Get all CVE that are mapped to a given CWE, search in given list 'li' '''
def get_cve_cwe(cwe, li):
    cves = []
    for item in li:
        if(item[2] == cwe):
            cves.append(item[0])
    return(cves)

In [32]:
cwe_dict = {}
for i, row in cwe_rels.iterrows():
    child = row['Child']
    parent = row['Parent']
    rel = row['Relationship']
    
    if rel == 'ChildOf':
        if child in cwe_dict:
            cwe_dict[child].append(parent)
        else:
            cwe_dict[child] = []
            cwe_dict[child].append(parent)
#print(cwe_dict)

In [33]:
# using df_cwe_cate or df_cwe_view for querying for CWEs in that list (that can be used to map CVEs i.e. in CWE-1003 list)

def get_cwe_subtree(cweid):
    cwe_members = []
    if cweid in df_cwe_cate['ID'].tolist():    
        item = df_cwe_cate.loc[df_cwe_cate['ID'] == cweid]
        cwe_str = item['Has_Member']
        cwe_members= cwe_str.str.split(';')
        
    elif cweid in df_cwe_view['ID'].tolist():  
        item = df_cwe_view.loc[df_cwe_view['ID'] == cweid]
        cwe_str = item['Has_Member']
        cwe_members= cwe_str.str.split(';')
     
    cwe_ret = []
    for item in cwe_members:
        children = get_cwe_children(item)
        if item in cwe_1003:
            cwe_ret.append(item)
        for ch in children:
            if ch in cwe_1003:
                cwe_ret.append(ch)
    return cwe_ret

In [34]:
# This is the first method I implemented to find family of the CWE. Usually, this yields a 
# large list of CWE that are related to each other, since we are returning the whole branch
# that CWE is in. Rather, we can also just return direct parents and children (below cell)

def get_cwe_parents(cwe):
    #given a cwe, find all related cwe that are in the same branch.
    family = []
    
    if cwe in cwe_dict:
        # exists as key - has parents.
        value = cwe_dict[cwe]
        for item in value:
            family.append(item)
            res = get_cwe_family(item) #recursively reach boss parent
            if res:
                family.extend(res)
    return family

def get_cwe_children(cwe):
    family = []
    
    if any(cwe in val for val in cwe_dict.values()):
        keys = [key for key, value in cwe_dict.items() if cwe in value]
        #print(keys)
        family.extend(keys)
        for item in keys:
            res = get_cwe_children(item)  
            if res:
                family.extend(res)
    return family
    
def get_cwe_family(cwe):
    return set(get_cwe_parents(cwe) + get_cwe_children(cwe))

In [35]:
query_list = []

In [36]:
for view in views:
    cve_list = get_cve_cwe(view[0], kg_cve2cwe)
    
    if(len(cve_list)>0):
        print(view[0], cve_list)
        query_list.append([view[0], cve_list])

In [37]:
for cate in categories:
    cve_list2 = get_cve_cwe(cate[0], kg_cve2cwe)
    
    if(len(cve_list2)>0):
        #print(cate[0], cve_list2)
        query_list.append([cate[0], cve_list2])

In [38]:
# List of discouraged CWE - so that these could be avoided when predicting good CWE possibilities
cwe_disc = ["CWE-119","CWE-20","CWE-200","CWE-269","CWE-289",
"CWE-311","CWE-330","CWE-345","CWE-74","CWE-400","CWE-610","CWE-662",
"CWE-665","CWE-668","CWE-682","CWE-697","CWE-755","CWE-834"]

In [43]:
# Filtering those views and categories with no members
cwe_nomember = []
for item in query_list:
    cwe_subtree = get_cwe_subtree(item[0])
    if len(cwe_subtree) == 0:
        #print(item[0])
        cwe_nomember.append(item[0])

In [44]:
# Combined list of PROHIBITED cwe = categories + views
cwe_proh = []
for cate in categories:
    cwe_proh.append(cate[0])
    
for view in views:
    cwe_proh.append(view[0])

In [45]:
# Query triples for Discouraged set - CWE and the list of CVE mapped to those
query_disc = {}
for weak in cwe_disc:
    query_disc[weak] = []

for row in kg_cvecwe_all:
    cwe = row[2]#'CWE']
    for i in cwe.split(';'):
        if i in cwe_disc:
            query_disc[i].append(row[0])

In [48]:
# Counting how many CVE are mapped to each Discouraged CWE in 2021. 
total = 0
for key in query_disc:
    print(key, len(query_disc[key]))
    total += len(query_disc[key])
print("total",total) #32761 for 2024 - total 30406 for 2021

CWE-119 11476
CWE-20 8389
CWE-200 6722
CWE-269 1109
CWE-289 0
CWE-311 169
CWE-330 141
CWE-345 163
CWE-74 516
CWE-400 912
CWE-610 55
CWE-662 23
CWE-665 155
CWE-668 225
CWE-682 52
CWE-697 21
CWE-755 219
CWE-834 59
total 30406


In [46]:
# Query triples for Discouraged set - CWE and the list of CVE mapped to those
query_proh = {}
for weak in cwe_proh:
    query_proh[weak] = []

for row in kg_cvecwe_all:
    cwe = row[2]
    for i in cwe.split(';'):
        if i in cwe_proh:
            query_proh[i].append(row[0])

In [47]:
# Counting how many CVE are mapped to each Prohibited CWE in 2021. 
total = 0
for key in query_proh:
    if len(query_proh[key]) > 0:
        print(key, len(query_proh[key]))
        total += len(query_proh[key])

print("total",total) #13180 for 2024, 13596 for 2021.

CWE-16 267
CWE-189 1297
CWE-19 229
CWE-199 6
CWE-254 421
CWE-255 742
CWE-264 5328
CWE-275 63
CWE-310 2494
CWE-320 41
CWE-361 7
CWE-371 1
CWE-388 35
CWE-399 2649
CWE-417 15
CWE-895 1
total 13596


In [49]:
# Updated CVE-CWE mappings as of 2024 December - list we use to 'verify'
new_cve_df = pd.read_csv('./cve2cwe_2024.csv')

Next we filter the query sets to get the mappings that were updated

In [50]:
# Prohibited Set
filtered_query_p = []
for key,value in query_proh.items():
    cwe_q = key
    cve_q = value
    cve_list = []
    for cve in cve_q: 
        row = new_cve_df.loc[new_cve_df["subject:START_ID"] == cve]
        cwe_match = list(row["object:END_ID"].values)
        #print(cwe_match)
        try:
            cwe_match.remove(cwe_q)
            # remove the known cwe from the updated CVE mapping, if it still exists
            # If the cwe was removed, this will fall into except
            # Otherwise, we'll include the new cwe and model will try to predict that 
        except:
            continue
        if len(cwe_match)>0:
            cve_list.append(cve)
    if cve_list:
        filtered_query_p.append([cwe_q,cve_list])

In [51]:
# Discouraged Set
filtered_query_d = []
for key,value in query_disc.items():
    cwe_q = key
    cve_q = value
    cve_list = []
    for cve in cve_q: 
        row = new_cve_df.loc[new_cve_df["subject:START_ID"] == cve]
        cwe_match = list(row["object:END_ID"].values)
        #print(cwe_match)
        try:
            cwe_match.remove(cwe_q)
        except:
            continue
        if len(cwe_match)>0:
            cve_list.append(cve)
    if cve_list:
        filtered_query_d.append([cwe_q,cve_list])

In [52]:
df_query = pd.DataFrame(filtered_query_p, columns = ['CWE','CVE'])
df_query.to_csv('filtered_query_p.csv')

In [53]:
df_query = pd.DataFrame(filtered_query_d, columns = ['CWE','CVE'])
df_query.to_csv('filtered_query_d.csv')

Here, we are done with our preprocessing code.
We filtered the CVE that were mapped to prohibited or discouraged CWE on 2021 August, and were updated by 2024 December. The filtered query files can be used in the FixV2W tutorial.